# Driverless AIモデルのMLOpsデプロイメント - Step1
- Driverless AIへ接続し、Projectの作成とExperimentの登録を実施

参考ワークフロー：https://docs.h2o.ai/mlops/py-client/py-client-examples/dai

In [1]:
import getpass

import h2osteam

h2osteam.__version__

'1.9.4'

In [2]:
import driverlessai
driverlessai.__version__

'1.10.5.post1'

## Driverless AIで作成済みExperimentをProject登録

SteamとDriverless AIへの接続に関して：　https://github.com/yukismd/H2O_AI_Cloud/blob/main/client_access_external/Client_Access_ModelBuild.ipynb

### Steam接続

In [3]:
# Steamへログイン（本セルを実行後、表示された入力ボックスへパスワード（トークン）を入力）
h2osteam.login(url="https://steam.internal.dedicated.h2o.ai", 
               username="yuki.shimada@h2o.ai",
               password=getpass.getpass('Enter your API token here'),
               verify_ssl=False)

Enter your API token here········


### Driverless AIへの接続とProject作成とExperiment登録

In [4]:
# 既存インスタンスの確認
h2osteam.clients.DriverlessClient().get_instances()

In [5]:
# 利用インスタンス
dai_instance = h2osteam.clients.DriverlessClient().get_instance(name="DAI-Steam-latest")
dai_instance

In [6]:
# インスタンスの状態
dai_instance.status()

'running'

In [7]:
# Driverless AI Clientの取得（インスタンスの状態がrunningである必要あり）
dai = dai_instance.connect()
dai

<class 'driverlessai._core.Client'> https://steam.internal.dedicated.h2o.ai:443/proxy/driverless/619

Driverless AI Python Client: https://docs.h2o.ai/driverless-ai/pyclient/docs/html/index.html

In [8]:
# Experimentのリスト
dai.experiments.list()

    | Type       | Key                                  | Name
----+------------+--------------------------------------+--------------
  0 | Experiment | 6e18315a-43ab-11ee-bd1a-4a8d6bd753d2 | model-iris-2
  1 | Experiment | d935b5b2-43aa-11ee-bd1a-4a8d6bd753d2 | model-iris-1

In [21]:
# 各Experimentオブジェクトの取得
mod1 = dai.experiments.list()[1]
mod2 = dai.experiments.list()[0]

mod1.name, mod2.name, type(mod1)

('model-iris-1', 'model-iris-2', driverlessai._experiments.Experiment)

In [22]:
# Projectのリスト
dai.projects.list()

    | Type    | Key                                  | Name
----+---------+--------------------------------------+-------------------
  0 | Project | 10ffe56c-6ef8-45c8-bc42-7aea251da2d1 | Proj-iris
  1 | Project | 297142df-702f-4212-ad83-3bed0218be1f | proj_boston
  2 | Project | dfc7fd0a-7645-4b7e-a082-b8846b032daa | car_obj_detection

In [25]:
# Projectの作成
new_project = dai.projects.create(
    name = 'Proj-iris',   # Project名
    #force = True    # create new project even if project with same name already exists
    force = False
)
new_project

In [27]:
dai.projects.list()
# Project（Proj-iris）が作成された

    | Type    | Key                                  | Name
----+---------+--------------------------------------+-------------------
  0 | Project | 91d6b769-9e9b-4606-8984-278b632716d1 | Proj-iris
  1 | Project | 297142df-702f-4212-ad83-3bed0218be1f | proj_boston
  2 | Project | dfc7fd0a-7645-4b7e-a082-b8846b032daa | car_obj_detection

In [39]:
# Project（Proj-iris）へのExperiment（model-iris-1 & model-iris-2）の登録
new_project.link_experiment(mod1)    # model-iris-1の登録
new_project.link_experiment(mod2)    # model-iris-2の登録

In [40]:
# Project（Proj-iris）へ登録されたExperiment
new_project.experiments

    | Type       | Key                                  | Name
----+------------+--------------------------------------+--------------
  0 | Experiment | 6e18315a-43ab-11ee-bd1a-4a8d6bd753d2 | model-iris-2
  1 | Experiment | d935b5b2-43aa-11ee-bd1a-4a8d6bd753d2 | model-iris-1

#### Driverless AI - 作成されたProject（Proj-iris）と、登録されたExperiment（Experiments数が2となっている）
<img src="image/Experiments-on-Project.png" width=900px>

#### MLOps - 連携されたProject（Proj-iris）と、登録されたExperiment（Experiment Countが2となっている）
<img src="image/MLOps-Projects.png" width=800px>